In [ ]:
# Install fork of spint with modifications to accept multiple cost variables
# !pip install git+https://github.com/chesterharvey/spint

In [1]:
from modeling import *
from feature_dev import *
from utils import configs

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_rows', None)

df = develop_features()
# df.columns.tolist()

8190 initial records
8190 complete records from load_travel_time_and_fares
8190 complete records from load_track_miles
8190 complete records from load_am_ridership
8190 complete records from load_pm_ridership
8190 complete records from load_off_peak_ridership
8190 complete records from calculate_total_ridership_and_parking_users
8190 complete records from load_am_auto_travel_time
8190 complete records from load_pm_auto_travel_time
8190 complete records from load_off_peak_auto_travel_time
8190 complete records from load_bus_travel_time
8190 complete records from load_auto_miles
8190 complete records from load_m25_stations
8190 complete records from load_median_hh_income
8190 complete records from load_nine_to_five_jobs
8190 complete records from load_night_and_weekend_jobs
8190 complete records from load_school_jobs
8190 complete records from load_sop
8190 complete records from calculate_sums_from_columns
8190 complete records from calculate_rates_from_constants
8190 complete records fr

In [ ]:
output_df, grav_model = estimate_models(df, cost_func='pow', **task_2_no_track_mile_model_spec)
output_df

In [ ]:
dir(grav_model)

In [ ]:
yhat = grav_model.yhat

In [ ]:
y = np.transpose(grav_model.y)[0]

In [ ]:
r = np.corrcoef(yhat, y)[0,1]

In [ ]:
r * r

Why not combine flows at times of day as units of analysis and then control for am, pm, and off peak and fixed effects?

Count flows within hours of the day (or maybe 3-hour chunks?) for each day of the week (or maybe weekdays/weekends?)

# Vars to investigate
- Parking spaces per station
- Land uses per station
    - Job types may be best proxy
- Percent of users who parked

In [ ]:
from flow_counts import *

In [ ]:
df = load_trips_record()

In [ ]:
df.head(2).T

In [ ]:
df.to_parquet(path='rail_trips.parquet')

In [ ]:
path = os.path.join(configs.box_data_dir, 'Data/railTrips_withBusParkingFlags_04152022-05152022.xlsx')
df = pd.read_excel(path)

In [ ]:
df.head().columns.tolist()

In [ ]:
df.head()

In [ ]:
df['day_of_week'] = df.START_TIME.dt.day_name().value_counts()

In [ ]:
df.START_TIME.iloc[0].time()

In [ ]:
breaks = [
        ('off peak', '05:00'),
        ('am', '12:00'),
        ('pm', '21:30'),
        ('off peak', '23:59'),
    ]
bins = [pd.to_datetime('00:00').time()] + [pd.to_datetime(x).time() for x in [b for (_, b) in breaks]]
minute_bins = [(x.hour * 60) + x.minute for x in bins]
labels = [a for (a, _) in breaks]
minute_col = df['START_TIME'].dt.hour * 60 + df['START_TIME'].dt.minute
df['start_time_category'] = pd.cut(minute_col, bins=minute_bins, labels=labels, include_lowest=True, ordered=False)

df.start_time_category.value_counts()

In [ ]:
[(x.hour * 60) + x.minute for x in bins]